In [17]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io

from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Dense, Conv2D, Flatten
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model

# Deep Learning study on the results of the 1D Pseudo-Wigner Distribution using Neural Networks

In this test, we are going to use the Wigner distribution of the datasets created by octave codes.

Some references for the CNN application with keras (python):
* https://towardsdatascience.com/building-a-convolutional-neural-network-cnn-in-keras-329fbbadc5f5 

In [18]:
# Laod the dataset

dataset = np.load('wd_results.npy')
dataset.shape

FileNotFoundError: [Errno 2] No such file or directory: 'wd_results.npy'

## CNN (Convolutional Neural Networks)

**Data pre-processing**

In [4]:
def compute_targets_array(nb_class, X_train):
    
    nb_holograms = X_train.shape[0]
    nb_holograms_class = int(nb_holograms / nb_class)
    
    # Y array
    Y_array = np.zeros((nb_holograms,))
    counter = 1
    target = 0 # 1
    
    for i in range(nb_holograms):
        if counter == (nb_holograms_class + 1):
            target = target + 1
            counter = 1
        Y_array[i,] = target
        counter = counter + 1    
    
    return Y_array

In [5]:
# Data pre-procesing
X_train = dataset[:,0,:,:]
X_train = X_train.reshape(25,200,200,1)
print(X_train.shape)

(25, 200, 200, 1)


In [6]:
# Compute array of targets
nb_class = 5
Y_array = compute_targets_array(nb_class, X_train)
print(Y_array)

# One-hot encode target column
Y_train = to_categorical(Y_array)
print(Y_train)

[0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 4. 4. 4. 4.
 4.]
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


**Building the model**

In [7]:
# Create model
model = Sequential() # allows build a model layer by layer

# Add model layers

# Conv2D layer: 
# 64 nodes, 3x3 filter matrix, Rectified Linear Activation as activation function,
# shape of each input (200, 200, 1,) with 1 signifying images are greyscale
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(200,200,1))) 

# 32 nodes
model.add(Conv2D(32, kernel_size=3, activation='relu'))

# Flatten layer: connection between the convolution and dense layers
model.add(Flatten())

# Dense layer: used for the output layer
# 5 nodes for the output layer, one for each possible outcome (1-5)
# 'softmax' as activation function, it makes the output sump up to 1 so the output
# can be interpreted as probalities
model.add(Dense(5, activation='softmax'))

**Compiling the model**

In [8]:
# Three parameters:
# optmizer: 'adam'
# loss function: 'categorical_crossentropy', the most common choice for classification
# metrics: 'accuracy', to see the accuracy score
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

**Training the model**

In [9]:
# Number of epochs: number of tmes the model wil cycle trough the data
model.fit(X_train, Y_train, validation_data=(X_train, Y_train), epochs=10)

Train on 25 samples, validate on 25 samples
Epoch 1/10
25/25 [==============================] - 5s 204ms/step - loss: 13574.5508 - accuracy: 0.2400 - val_loss: 2178413.0000 - val_accuracy: 0.4000
Epoch 2/10
25/25 [==============================] - 5s 184ms/step - loss: 2178413.2500 - accuracy: 0.4000 - val_loss: 1573390.7500 - val_accuracy: 0.2800
Epoch 3/10
25/25 [==============================] - 5s 181ms/step - loss: 1573390.7500 - accuracy: 0.2800 - val_loss: 996428.9375 - val_accuracy: 0.3200
Epoch 4/10
25/25 [==============================] - 5s 182ms/step - loss: 996428.9375 - accuracy: 0.3200 - val_loss: 599865.6875 - val_accuracy: 0.4000
Epoch 5/10
25/25 [==============================] - 5s 185ms/step - loss: 599865.6875 - accuracy: 0.4000 - val_loss: 332944.6562 - val_accuracy: 0.4000
Epoch 6/10
25/25 [==============================] - 5s 186ms/step - loss: 332944.6562 - accuracy: 0.4000 - val_loss: 111936.1797 - val_accuracy: 0.4400
Epoch 7/10
25/25 [=======================

**Evalutation**

In [10]:
# Evaluate the keras model
_, accuracy = model.evaluate(X_train, Y_train, verbose=0)
print('Accuracy: %.2f%%' % (accuracy*100))

Accuracy: 100.00%


**Make predictions**

In [11]:
# Make probability predictions with the model
predictions = model.predict(X_train)

# Round predictions 
rounded = [round(x[0]) for x in predictions]

# Make class predictions with the model
predictions = model.predict_classes(X_train)

# Summarize the first 5 cases
for i in range(5):
    print('%d (expected %d)' % (predictions[i], Y_array[i]))

0 (expected 0)
0 (expected 0)
0 (expected 0)
0 (expected 0)
0 (expected 0)


**Save weights and model**

In [20]:
# Serialize model to JSON
model_json = model.to_json()
with open("output/neural_networks/model.json", "w") as json_file:
    json_file.write(model_json)
    
# Serialize weights to HDF5
model.save_weights("output/neural_networks/model.h5")
print("Saved model structure and weights")

Saved model structure and weights


**Load model**

In [13]:
'''
# The model weights and architecture were saved separated, so it must re-compile

# Load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# Load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# Evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(X_train, Y_train, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
'''

'\n# The model weights and architecture were saved separated, so it must re-compile\n\n# Load json and create model\njson_file = open(\'model.json\', \'r\')\nloaded_model_json = json_file.read()\njson_file.close()\nloaded_model = model_from_json(loaded_model_json)\n\n# Load weights into new model\nloaded_model.load_weights("model.h5")\nprint("Loaded model from disk")\n \n# Evaluate loaded model on test data\nloaded_model.compile(loss=\'categorical_crossentropy\', optimizer=\'adam\', metrics=[\'accuracy\'])\nscore = loaded_model.evaluate(X_train, Y_train, verbose=0)\nprint("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))\n'

**Summary**

In [14]:
# Summarize model.
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 198, 198, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 196, 196, 32)      18464     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1229312)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6146565   
Total params: 6,165,669
Trainable params: 6,165,669
Non-trainable params: 0
_________________________________________________________________


**Plot model**

In [16]:
# Error

# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

## Brouillon

In [ ]:
# Convolutional Neural Network
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
visible = Input(shape=(64,64,1))
conv1 = Conv2D(32, kernel_size=4, activation='relu')(visible)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(16, kernel_size=4, activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
flat = Flatten()(pool2)
hidden1 = Dense(10, activation='relu')(flat)
output = Dense(1, activation='sigmoid')(hidden1)
model = Model(inputs=visible, outputs=output)
# summarize layers
print(model.summary())
# plot graph
plot_model(model, to_file='convolutional_neural_network.png')